In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
! pip install glove-python-binary

     |████████████████████████████████| 948 kB 8.3 MB/s 


In [16]:
from glove import Glove
glove=Glove()
glove_model=glove.load("/content/gdrive/MyDrive/FYP/CAA_NRC/Glove/glove.model")

In [17]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/gdrive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=pd.get_dummies(df["label"])
df

,label,new_text
0,0,muslim immigrant says that we will conquer ger...
1,1,my message to indian muslims kashmir is the sp...
2,2,normally never i abuse anyone on twitter but t...
3,1,heard someone suggesting holding anti caa nrc ...
4,2,the apex court has issued a notice to the unio...
...,...,...
3096,2,save hindus infiltrators anti national go to p...
3097,2,breaking bhimarmychief chandrashekhar azad has...
3098,0,tamil nadu cm edappadi k palaniswamy says citi...
3099,2,some more photos of protest meetings against c...


In [18]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [19]:

import tensorflow as tf
maxlen=maxi
max_words=len(glove_model.dictionary)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [20]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in glove_model.dictionary:
    embed_vector=glove_model.word_vectors[glove_model.dictionary[word]]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [21]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.10990729,  0.04230337,  0.05654864, ..., -0.20832376,
        -0.03223338, -0.21452594],
       [ 0.08457289, -0.06926741, -0.00822759, ..., -0.15099157,
        -0.02485671, -0.19643595],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=maxi,embeddings_initializer=Constant(embed_matrix)))
model.add(GRU(128,return_sequences=True))
model.add(GRU(128,return_sequences=False)) 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(3,activation='softmax'))

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
37/37 [==============================] - 3s 14ms/step - loss: 1.0895 - accuracy: 0.3987
Epoch 2/10
37/37 [==============================] - 0s 12ms/step - loss: 1.0721 - accuracy: 0.4215
Epoch 3/10
37/37 [==============================] - 0s 12ms/step - loss: 1.0315 - accuracy: 0.4787
Epoch 4/10
37/37 [==============================] - 0s 12ms/step - loss: 0.9241 - accuracy: 0.5871
Epoch 5/10
37/37 [==============================] - 0s 11ms/step - loss: 0.7466 - accuracy: 0.6611
Epoch 6/10
37/37 [==============================] - 0s 11ms/step - loss: 0.6168 - accuracy: 0.7191
Epoch 7/10
37/37 [==============================] - 0s 11ms/step - loss: 0.4864 - accuracy: 0.7531
Epoch 8/10
37/37 [==============================] - 0s 12ms/step - loss: 0.4156 - accuracy: 0.7974
Epoch 9/10
37/37 [==============================] - 0s 11ms/step - loss: 0.3292 - accuracy: 0.8370
Epoch 10/10
37/37 [==============================] - 0s 12ms/step - loss: 0.2871 - accuracy: 0.8675


In [25]:
# y_pred=model.predict(X_test)
# for i in range(len(y_pred)):
#   if y_pred[i]>0.5:
#     y_pred[i]=1
#   else:
#     y_pred[i]=0

In [26]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
# display_metrics(y_test,y_pred)

In [27]:
import numpy as np
y_pred=model.predict(X_test)
y_test=y_test.to_numpy()
l=[np.argmax(i) for i in y_pred]
M=[np.argmax(i) for i in y_test]
display_metrics(M,l)

0.5992268041237113
0.5852776425226338
0.5859244689187213
0.5996337580735464
